<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/PAXDEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dateutil faker -q
!pip install geopy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.0 MB/s eta 0:00:00


In [ ]:
import os

if __name__ == "__main__":
    # Delete the database file if it exists
    database_file = 'recoverai.db'
    if os.path.exists(database_file):
        try:
            os.remove(database_file)
            print(f"RecoverAI Agent: Existing database file '{database_file}' deleted.")
        except OSError as e:
            print(f"RecoverAI Agent: Error deleting database file '{database_file}': {e}")
            # Consider raising the exception or handling it appropriately based on your requirements
    else:
        print(f"RecoverAI Agent: Database file '{database_file}' not found.")


import random
import sqlite3
from datetime import datetime, timedelta
from faker import Faker

# --- RecoverAI (RAI) Profile ---
RAI_PROFILE = {
    "airline_name": "Sun Airlines",
    "aircraft_types": ["A320", "B737", "B787", "B777"],
    "crew_roles": ["CP", "FO", "FA"],  # Captain, First Officer, Flight Attendant
    "connection_times": {
        "domestic": timedelta(minutes=15),
        "international": timedelta(minutes=30)
    },
    "long_haul_augmented_crew": True,  # Details to be defined later
    "reserve_crew_and_aircraft": True,
    "crew_round_trip": True,
    "protect_pnr": True,
    "pnr_seat_assignment": True,
    "airports": {
        "domestic": ["JFK", "LAX", "ORD", "ATL", "DFW", "DEN", "SFO", "SEA", "MIA"],
        "international": ["HND", "ICN", "PVG", "HKG", "SIN", "LHR", "CDG", "FRA", "AMS", "MAD", "CUN", "PUJ", "MBJ", "SJU", "NAS"]
    },
    "home_bases": {
        "domestic": "DEN",
        "asia": "PVG",
        "europe": "FRA",
        "caribbean": "CUN"
    },
    "maintenance_bases": ["DEN", "JFK", "ATL", "ORD", "PVG", "FRA", "CUN"],
    "capacities": {
        "aircraft": {
            "A320": {"BC": 8, "EC": 150},
            "B737": {"BC": 10, "EC": 162},
            "B787": {"BC": 28, "EC": 269},
            "B777": {"BC": 40, "EC": 350}
        },
        "crew": {
            "A320": {"CP": 1, "FO": 1, "FA": 3},
            "B737": {"CP": 1, "FO": 1, "FA": 4},
            "B787": {"CP": 1, "FO": 1, "FA": 6},
            "B777": {"CP": 1, "FO": 1, "FA": 8}
        }
    }
}


# --- Database Setup and Data Generation ---

def create_tables():
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()


    cursor.execute("""
        CREATE TABLE IF NOT EXISTS flights (
            flight_number TEXT PRIMARY KEY,
            origin TEXT,
            destination TEXT,
            departure_time DATETIME,
            arrival_time DATETIME,
            aircraft_id TEXT,
            status TEXT,
            cost REAL,
            distance REAL  -- Add the distance column here
        )
    """)

    # OAG flights
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS oag_flights (
            flight_number TEXT PRIMARY KEY,
            origin TEXT,
            destination TEXT,
            departure_time DATETIME,
            arrival_time DATETIME,
            aircraft_type TEXT,  -- Added aircraft_type column
            status TEXT,
            cost REAL,
            distance REAL
        )
    """)

    cursor.execute("""
       CREATE TABLE IF NOT EXISTS passengers (
           passenger_id INTEGER PRIMARY KEY AUTOINCREMENT,
           name TEXT,
           flight_number TEXT,
           seat_number TEXT,
           pnr TEXT,
           class_of_service TEXT,
           ticket_type TEXT,
           FOREIGN KEY (flight_number) REFERENCES flights(flight_number)
       )
   """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS crew (
            crew_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            role TEXT,
            flight_number TEXT,
            is_reserve BOOLEAN,
            FOREIGN KEY (flight_number) REFERENCES flights(flight_number)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS seat_assignments (
            seat_assignment_id INTEGER PRIMARY KEY AUTOINCREMENT,
            flight_number TEXT,
            seat_number TEXT,
            passenger_id INTEGER,
            class_of_service TEXT,
            FOREIGN KEY (flight_number) REFERENCES flights(flight_number),
            FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS aircraft (
            aircraft_id TEXT PRIMARY KEY,
            type TEXT,
            capacity INTEGER,
            is_reserve BOOLEAN
        )
    """)

    conn.commit()
    conn.close()

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
import functools

@functools.lru_cache(maxsize=128)  # Cache results for up to 128 locations
def calculate_distance(origin, destination):
    """Calculates the distance between two airports using GeoPy with rate limiting and caching."""
    geolocator = Nominatim(user_agent="recoverai", timeout=10)  # Increased timeout to 10 seconds
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # Rate limiting: 1 second between requests

    try:
        location_origin = geocode(origin)
        location_destination = geocode(destination)

        if location_origin and location_destination:
            distance = geodesic((location_origin.latitude, location_origin.longitude),
                                (location_destination.latitude, location_destination.longitude)).km
            return distance
        else:
            print(f"RecoverAI Agent: Could not find location data for {origin} or {destination}.")
            return None  # Or handle the error appropriately

    except Exception as e:
        print(f"RecoverAI Agent: Error calculating distance: {e}")
        return None  # Or handle the error appropriately

def generate_data():
    """Generates sample data for flights, passengers, crew, and aircraft."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()
    faker = Faker()

    # --- Generate aircraft ---
    existing_aircraft_ids = set()
    for i in range(50):
        while True:
            aircraft_type = random.choice(RAI_PROFILE["aircraft_types"])
            aircraft_id = f"{aircraft_type}-{random.randint(1, 50):03}"
            if aircraft_id not in existing_aircraft_ids:
                existing_aircraft_ids.add(aircraft_id)
                break

        capacity = RAI_PROFILE["capacities"]["aircraft"][aircraft_type]["EC"] + RAI_PROFILE["capacities"]["aircraft"][aircraft_type]["BC"]
        # Some aircraft are designated as reserves
        is_reserve = i % 10 == 0  # Example: Every 10th aircraft is a reserve

        cursor.execute("INSERT INTO aircraft VALUES (?, ?, ?, ?)", (aircraft_id, aircraft_type, capacity, is_reserve))


    # --- Generate OAG flights with random airline codes ---
    airline_codes = ["AA", "DL", "UA"]
    existing_flight_numbers = set()

    for i in range(200):
        while True:
            flight_number = f"{random.choice(airline_codes)}{random.randint(1, 9999):04}"
            if flight_number not in existing_flight_numbers:
                existing_flight_numbers.add(flight_number)
                break

        origin = random.choice(RAI_PROFILE["airports"]["domestic"] + RAI_PROFILE["airports"]["international"])
        destination = random.choice([a for a in RAI_PROFILE["airports"]["domestic"] + RAI_PROFILE["airports"]["international"] if a != origin])
        departure_time = faker.date_time_between(start_date="-1d", end_date="+1d")
        arrival_time = departure_time + timedelta(hours=random.randint(2, 12))

        # Get a random aircraft_id from the aircraft table or choose a random aircraft type:
        # Option 1: Get a random aircraft_id
        # cursor.execute("SELECT aircraft_id FROM aircraft ORDER BY RANDOM() LIMIT 1")
        # aircraft_id = cursor.fetchone()[0]
        # aircraft_type = aircraft_id.split('-')[0]  # Extract aircraft type from aircraft_id

        # Option 2: Choose a random aircraft type directly
        aircraft_type = random.choice(RAI_PROFILE["aircraft_types"])

        # Calculate distance
        distance = calculate_distance(origin, destination)
        if distance is None:
            distance = 0  # Handle cases where distance calculation fails

        # Calculate cost (using calculate_flight_cost or your existing logic)
        # Example using calculate_flight_cost:
        flight_data = {'aircraft_type': aircraft_type, 'distance': distance}  # Replace with actual flight data if needed
        cost = calculate_flight_cost(flight_data)  # Call your cost calculation function


        status = "Scheduled"


        is_domestic = origin in RAI_PROFILE["airports"]["domestic"] and destination in RAI_PROFILE["airports"]["domestic"]

        # Select a random aircraft type based on flight type (domestic or international)
        if is_domestic:
            cursor.execute("SELECT type FROM aircraft WHERE type IN ('A320', 'B737') ORDER BY RANDOM() LIMIT 1")
        else:
            cursor.execute("SELECT type FROM aircraft WHERE type IN ('B787', 'B777') ORDER BY RANDOM() LIMIT 1")

        aircraft_type = cursor.fetchone()[0]

        # Insert into oag_flights with aircraft_type
        cursor.execute("INSERT INTO oag_flights VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                      (flight_number, origin, destination, departure_time, arrival_time, aircraft_type, status, cost, distance))







    # --- Generate flights ---
    cursor.execute("SELECT aircraft_id FROM aircraft WHERE is_reserve = 0")  # Select non-reserve aircraft
    available_aircraft_ids = [row[0] for row in cursor.fetchall()]

    for i in range(100):
        flight_number = f"SA{i+1:03}"
        origin = random.choice(RAI_PROFILE["airports"]["domestic"] + RAI_PROFILE["airports"]["international"])
        destination = random.choice([a for a in RAI_PROFILE["airports"]["domestic"] + RAI_PROFILE["airports"]["international"] if a != origin])
        departure_time = faker.date_time_between(start_date="-1d", end_date="+1d")
        arrival_time = departure_time + timedelta(hours=random.randint(2, 12))  # Adjust flight duration as needed

        # Assign aircraft_id from available_aircraft_ids
        aircraft_id = random.choice(available_aircraft_ids)

        status = "Scheduled"

        # Calculate cost
        distance = random.randint(500, 5000)  # Placeholder for initial distance
        fuel_cost_per_unit = 0.15
        crew_cost_per_hour = 100

        # Get aircraft type from aircraft_id
        aircraft_type = aircraft_id.split('-')[0]

        num_crew = sum(RAI_PROFILE["capacities"]["crew"].get(aircraft_type, {}).values())
        flight_duration_hours = (arrival_time - departure_time).total_seconds() / 3600
        cost = distance * fuel_cost_per_unit + num_crew * crew_cost_per_hour * flight_duration_hours

        # Calculate distance using GeoPy
        distance = calculate_distance(origin, destination)
        if distance is None:
            distance = 0  # Handle cases where distance calculation fails by setting a default value

        cursor.execute("INSERT INTO flights VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                      (flight_number, origin, destination, departure_time, arrival_time, aircraft_id, status, cost, distance))


    # --- Generate passengers ---
    for i in range(500):  # Or however many passengers you are generating
        name = faker.name()
        flight_number = f"SA{random.randint(1, 100):03}"
        pnr = faker.bothify(text='?????-#####')
        class_of_service = random.choice(["EC", "BC"])
        ticket_type = "R"  # Example of adding a ticket_type value

        #print(f"About to insert passenger: {name}, {flight_number}, {None}, {pnr}, {class_of_service}, {ticket_type}")

        cursor.execute("INSERT INTO passengers VALUES (?, ?, ?, ?, ?, ?, ?)", (None, name, flight_number, None, pnr, class_of_service, ticket_type))



    # --- Generate crew ---
    roles = RAI_PROFILE["crew_roles"]
    for i in range(200):
        name = faker.name()
        role = random.choice(roles)
        flight_number = f"SA{random.randint(1, 100):03}"  # Assign crew to flights
        is_reserve = False  # Initially, all crew are not reserves

        # Assign some crew as reserves based on a percentage
        if random.random() < 0.1:  # 10% chance of being a reserve
            is_reserve = True
            flight_number = None  # Reserves are not assigned to a specific flight

        cursor.execute("INSERT INTO crew VALUES (?, ?, ?, ?, ?)", (None, name, role, flight_number, is_reserve))

    # --- Generate seat assignments ---
    cursor.execute("SELECT passenger_id, flight_number, class_of_service FROM passengers")
    passengers_data = cursor.fetchall()

    for passenger in passengers_data:
        passenger_id, flight_number, class_of_service = passenger

        aircraft_type_result = cursor.execute("""
            SELECT T2.type
            FROM flights AS T1
            INNER JOIN aircraft AS T2 ON T1.aircraft_id = T2.aircraft_id
            WHERE T1.flight_number = ?
        """, (flight_number,)).fetchone()

        if aircraft_type_result:
            aircraft_type = aircraft_type_result[0]
        else:
            print(f"Skipping seat assignment for passenger {passenger_id} on flight {flight_number} (Aircraft type not found)")
            continue  # Skip to the next passenger

        while True:
            if class_of_service == "BC":
                seat_number = f"{random.randint(1, RAI_PROFILE['capacities']['aircraft'][aircraft_type]['BC']):02}{chr(random.randint(65, 68))}"
            else:
                seat_number = f"{random.randint(1, RAI_PROFILE['capacities']['aircraft'][aircraft_type]['EC']):02}{chr(random.randint(65, 70))}"

            if not cursor.execute("SELECT 1 FROM seat_assignments WHERE flight_number = ? AND seat_number = ?", (flight_number, seat_number)).fetchone():
                break

        cursor.execute("INSERT INTO seat_assignments (flight_number, seat_number, passenger_id, class_of_service) VALUES (?, ?, ?, ?)",
                       (flight_number, seat_number, passenger_id, class_of_service))

    conn.commit()  # Commit the changes to the database
    conn.close()

# --- Cost Calculation and Resource Management ---

def calculate_flight_cost(flight_data):
    """Calculates the total cost for a flight."""
    aircraft_type = flight_data["aircraft_type"]
    distance = flight_data["distance"]

    # Fuel cost (simplified example)
    fuel_cost_per_unit = {
        "A320": 0.15,
        "B737": 0.18,
        "B787": 0.22,
        "B777": 0.25
    }
    fuel_cost = distance * fuel_cost_per_unit[aircraft_type]

    #print(f"Fuel cost for {aircraft_type} aircraft: {fuel_cost}")

    # Crew cost
    crew_cost_per_hour = 100
    num_crew = sum(RAI_PROFILE["capacities"]["crew"].get(aircraft_type, {}).values())  # Handle missing aircraft type
    flight_duration_hours = flight_data.get("flight_duration_hours", 5)  # Default to 5 hours if not provided
    crew_cost = num_crew * crew_cost_per_hour * flight_duration_hours


    # Airport fees (simplified example)
    airport_fees = 5000  # Placeholder airport fees

    # Maintenance cost (simplified example)
    maintenance_cost = 2000  # Placeholder maintenance cost

    total_cost = fuel_cost + crew_cost + airport_fees + maintenance_cost
    return total_cost

def assign_crew_and_aircraft(flight):
    """Assigns crew and aircraft to a flight, considering reserves and round-trips."""
    aircraft_type = flight["aircraft_type"]
    required_crew = RAI_PROFILE["capacities"]["crew"][aircraft_type]

    # Simplified crew assignment (replace with more detailed logic)
    crew = []
    for role, count in required_crew.items():
        for _ in range(count):
            # Find available crew with matching role (consider reserves and round-trips)
            # ... (Implement your crew selection logic here) ...
            crew.append({"name": "Crew Member", "role": role})  # Placeholder crew member

    # Simplified aircraft assignment (replace with more detailed logic)
    aircraft_id = f"{aircraft_type}-001"  # Placeholder aircraft ID
    # Check if aircraft is available and not under maintenance
    # ... (Implement your aircraft selection logic here) ...

    flight["crew"] = crew
    flight["aircraft_id"] = aircraft_id
    print(f"Assigned crew and aircraft to flight {flight['flight_number']}: {flight}")

def update_flight_schedule(flight):
    """Updates the flight schedule, considering connection times and airport constraints."""
    # ... (Implement your schedule update logic here) ...
    # Adjust departure and arrival times based on delays, cancellations, etc.
    # Consider connection times, airport curfews, and other constraints.
    print(f"Updated schedule for flight {flight['flight_number']}: {flight}")

# --- Helper Functions ---

def update_flight_status(flight_number, status):
    """Updates the status of a flight in the database."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE flights SET status = ? WHERE flight_number = ?", (status, flight_number))
    conn.commit()
    conn.close()

def get_passengers_on_flight(flight_number):
    """Retrieves passengers assigned to a flight."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM passengers WHERE flight_number = ?", (flight_number,))
    passengers = cursor.fetchall()
    conn.close()
    return passengers


#-----NEW-BEGIN--

def sanitize_input(input_string):
    """Basic sanitization function to prevent SQL injection."""
    return input_string.replace("'", "''")

def get_aircraft_type(aircraft_id):
    """Helper function to retrieve aircraft type from the database."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()
    cursor.execute("SELECT type FROM aircraft WHERE aircraft_id = ?", (aircraft_id,))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else "A320"  # Default to A320 if not found


from datetime import datetime, timedelta
import sqlite3

def get_passengers_on_flight_with_details(flight_number):
    """Retrieves passenger details for a given flight."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()

    cursor.execute("""
        SELECT passenger_id, name, class_of_service
        FROM passengers
        WHERE flight_number = ?
    """, (flight_number,))
    passengers = cursor.fetchall()
    conn.close()
    return passengers

from datetime import datetime, timedelta
import sqlite3

def find_alternative_flight(passenger, original_flight):
    """Finds an alternative flight, prioritizing origin, connections, capacity, and reserves,
       considering both Sun Airlines and OAG flights.
    """
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()

    # Get details of all passengers on the original flight:
    original_passengers = get_passengers_on_flight_with_details(original_flight['flight_number'])

    # Sanitize input values
    origin = sanitize_input(original_flight['origin'])
    destination = sanitize_input(original_flight['destination'])
    departure_time_str = sanitize_input(str(original_flight['departure_time']))
    arrival_time_str = sanitize_input(str(original_flight['arrival_time'] + RAI_PROFILE['connection_times']['domestic']))
    end_time_str = sanitize_input(str(original_flight['arrival_time'] + timedelta(hours=48)))

    # 1. Query for suitable Sun Airlines alternative flights
    query_sun_airlines = f"""
        SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_id, cost, distance
        FROM flights
        WHERE destination = '{destination}'
          AND departure_time BETWEEN '{arrival_time_str}' AND '{end_time_str}'
          AND status = 'Scheduled'
    """
    cursor.execute(query_sun_airlines)
    suitable_alternatives = [
        {'flight_number': row[0], 'origin': row[1], 'destination': row[2],
         'departure_time': datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f"),
         'arrival_time': datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f"),
         'aircraft_id': row[5], 'cost': row[6], 'distance': row[7], 'is_oag': False,
         'aircraft_type': get_aircraft_type(row[5])}
        for row in cursor.fetchall()
    ]

    # Calculate cost for Sun Airlines alternatives (if not already present)
    for flight in suitable_alternatives:
        if 'calculated_cost' not in flight and not flight.get('is_oag', False):
            if flight.get('aircraft_id'):
                cursor.execute(f"SELECT type FROM aircraft WHERE aircraft_id = '{flight['aircraft_id']}'")
                aircraft_type_result = cursor.fetchone()
                aircraft_type = aircraft_type_result[0] if aircraft_type_result else "A320"
            else:
                aircraft_type = flight.get('aircraft_type', "A320")

            flight_data = {'aircraft_type': aircraft_type, 'distance': flight['distance']}
            flight['calculated_cost'] = calculate_flight_cost(flight_data)

    # 2. Prioritize flights from the original origin, then by cost, departure time, and origin distance
    origin_priority_alternatives = [
        flight for flight in suitable_alternatives
        if flight['origin'] == original_flight['origin']
    ]
    if origin_priority_alternatives:
        suitable_alternatives = origin_priority_alternatives
    else:
        for flight in suitable_alternatives:
            flight['origin_distance'] = calculate_distance(original_flight['origin'], flight['origin'])
        suitable_alternatives.sort(key=lambda flight: (flight['calculated_cost'], flight['departure_time'], flight['origin_distance']))

    # 3. Consider OAG flights
    print('OAG')
    if not suitable_alternatives:
        try:
            query_oag = f"""
                SELECT flight_number, origin, destination, departure_time, arrival_time,
                       CASE WHEN aircraft_type IS NULL THEN '' ELSE aircraft_type END AS aircraft_type,
                       cost, distance
                FROM oag_flights
                WHERE origin = '{origin}' AND destination = '{destination}' AND status = 'Scheduled'
                ORDER BY ABS(strftime('%s', departure_time) - strftime('%s', '{departure_time_str}'))
                LIMIT 10
            """
            cursor.execute(query_oag)

            oag_alternatives = [
                {'flight_number': row[0], 'origin': row[1], 'destination': row[2],
                 'departure_time': datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f"),
                 'arrival_time': datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f"),
                 'aircraft_type': row[5] if row[5] else "A320",
                 'cost': row[6], 'distance': row[7], 'is_oag': True}
                for row in cursor.fetchall()
            ]

            # Calculate cost and distance for OAG alternatives:
            for flight in oag_alternatives:
                if 'distance' not in flight or flight['distance'] is None:
                    flight['distance'] = calculate_distance(flight['origin'], flight['destination'])
                flight_data = {'aircraft_type': flight.get('aircraft_type'), 'distance': flight['distance']}
                flight['calculated_cost'] = calculate_flight_cost(flight_data)

            suitable_alternatives.extend(oag_alternatives)

        except sqlite3.Error as e:
            print(f"RecoverAI Agent: Error querying OAG flights: {e}")

    # 4. Consider Sun Airlines connection flights
    if not suitable_alternatives:
        hub_airports = RAI_PROFILE["airports"]["domestic"] + RAI_PROFILE["airports"]["international"]
        connection_time = timedelta(hours=2)

        for hub in hub_airports:
            cursor.execute("""
                SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_id, cost, distance
                FROM flights
                WHERE origin = ? AND destination = ? AND status = 'Scheduled'
                AND departure_time BETWEEN ? AND ?
            """, (original_flight['origin'], hub, original_flight['arrival_time'], original_flight['arrival_time'] + timedelta(hours=24)))
            first_legs = cursor.fetchall()

            for first_leg_data in first_legs:
                first_leg = {
                    'flight_number': first_leg_data[0], 'origin': first_leg_data[1], 'destination': first_leg_data[2],
                    'departure_time': datetime.strptime(first_leg_data[3], "%Y-%m-%d %H:%M:%S.%f"),
                    'arrival_time': datetime.strptime(first_leg_data[4], "%Y-%m-%d %H:%M:%S.%f"),
                    'aircraft_id': first_leg_data[5], 'cost': first_leg_data[6], 'distance': first_leg_data[7],
                    'aircraft_type': get_aircraft_type(first_leg_data[5])
                }

                cursor.execute("""
                    SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_id, cost, distance
                    FROM flights
                    WHERE origin = ? AND destination = ? AND status = 'Scheduled'
                    AND departure_time BETWEEN ? AND ?
                """, (hub, original_flight['destination'], first_leg['arrival_time'] + connection_time, first_leg['arrival_time'] + connection_time + timedelta(hours=12)))
                second_legs = cursor.fetchall()

                for second_leg_data in second_legs:
                    second_leg = {
                        'flight_number': second_leg_data[0], 'origin': second_leg_data[1], 'destination': second_leg_data[2],
                        'departure_time': datetime.strptime(second_leg_data[3], "%Y-%m-%d %H:%M:%S.%f"),
                        'arrival_time': datetime.strptime(second_leg_data[4], "%Y-%m-%d %H:%M:%S.%f"),
                        'aircraft_id': second_leg_data[5], 'cost': second_leg_data[6], 'distance': second_leg_data[7],
                        'aircraft_type': get_aircraft_type(second_leg_data[5])
                    }

                    total_cost = first_leg['cost'] + second_leg['cost']

                    suitable_alternatives.append({
                        "first_leg": first_leg,
                        "second_leg": second_leg,
                        "is_connection": True,
                        "total_cost": total_cost
                    })

    # 5. Filter alternatives based on capacity
    filtered_alternatives = []
    for flight in suitable_alternatives:
        if flight.get("is_connection"):
            aircraft_id = flight["first_leg"]["aircraft_id"]
        else:
            aircraft_id = flight["aircraft_id"]

        cursor.execute("SELECT capacity FROM aircraft WHERE aircraft_id = ?", (aircraft_id,))
        capacity_result = cursor.fetchone()
        capacity = capacity_result[0] if capacity_result else 0

        cursor.execute("SELECT COUNT(*) FROM passengers WHERE flight_number = ?", (original_flight['flight_number'],))
        passenger_count = cursor.fetchone()[0]

        if capacity >= passenger_count:
            filtered_alternatives.append(flight)

    suitable_alternatives = filtered_alternatives

    # 6. Consider reserve flights
    if not suitable_alternatives:
        cursor.execute("""
            SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_id, cost, distance
            FROM flights
            WHERE destination = ? AND departure_time >= ? AND status = 'Reserve'
        """, (original_flight['destination'], original_flight['arrival_time']))
        reserve_flights = [
            {'flight_number': row[0], 'origin': row[1], 'destination': row[2],
             'departure_time': datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f"),
             'arrival_time': datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f"),
             'aircraft_id': row[5], 'cost': row[6], 'distance': row[7], 'is_oag': False,
             'aircraft_type': get_aircraft_type(row[5])}
            for row in cursor.fetchall()
        ]

        if reserve_flights:
            reserve_flight = reserve_flights[0]
            update_flight_status(reserve_flight['flight_number'], "Scheduled")
            return reserve_flight


     # 7. Final prioritization
    if suitable_alternatives:
        suitable_alternatives.sort(key=lambda flight: (flight.get('total_cost', flight.get('cost')), flight.get('departure_time', flight.get("first_leg", {}).get('departure_time', flight.get('departure_time'))), flight.get('origin_distance', 0)))

    conn.close()
    return None if not suitable_alternatives else suitable_alternatives[0]

#----END------NEW-


def update_passenger_booking(passenger, new_flight):
    """Updates a passenger's booking with a new flight."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE passengers SET flight_number = ? WHERE passenger_id = ?", (new_flight['flight_number'], passenger[0]))
    conn.commit()
    conn.close()
    #print(f"Updated booking for {passenger[1]} to flight {new_flight['flight_number']}")
    print('\n')


     # --- Added: Printing flight details ---
    print(f"Updated booking for {passenger[1]} to flight {new_flight['flight_number']}")
    print(f"Departure Airport: {new_flight['origin']}, Departure Time: {new_flight['departure_time'].strftime('%Y-%m-%d %H:%M')}")
    print(f"Arrival Airport: {new_flight['destination']}, Arrival Time: {new_flight['arrival_time'].strftime('%Y-%m-%d %H:%M')}")


def generate_notification(passenger, flight):
    """Generates a notification for a passenger."""
    # ... (Implement your notification logic here) ...
    # This function would likely generate a message with the updated flight details
    # and send it to the passenger via email, SMS, or other communication channels.
    print(f"Notification for {passenger[1]}: Your new flight is {flight['flight_number']} departing at {flight['departure_time'].strftime('%Y-%m-%d %H:%M')}")

# --- Data Validation Functions ---

def validate_flight(flight):
    """Validates flight data for consistency."""
    errors = []
    if not flight['flight_number']:
        errors.append("Flight number is missing.")
    if flight['origin'] == flight['destination']:
        errors.append("Origin and destination cannot be the same.")
    # ... (Add other validation checks based on RAI_PROFILE) ...
    return errors



def validate_passenger(passenger):
    """Validates passenger data for consistency."""
    errors = []
    if not passenger['name']:
        errors.append("Passenger name is missing.")
    if not passenger['flight_number']:
        errors.append("Passenger flight number is missing.")
    # ... (Add other passenger validation checks based on RAI_PROFILE and your requirements) ...
    return errors

def validate_crew_member(crew_member):
    """Validates crew member data for consistency."""
    errors = []
    if not crew_member['name']:
        errors.append("Crew member name is missing.")
    if not crew_member['role']:
        errors.append("Crew member role is missing.")
    # ... (Add other crew member validation checks based on RAI_PROFILE and your requirements) ...
    return errors

def validate_seat_assignment(seat_assignment):
    """Validates seat assignment data for consistency."""
    errors = []
    if not seat_assignment['flight_number']:
        errors.append("Flight number is missing for seat assignment.")
    if not seat_assignment['seat_number']:
        errors.append("Seat number is missing for seat assignment.")
    if not seat_assignment['passenger_id']:
        errors.append("Passenger ID is missing for seat assignment.")
    # ... (Add other seat assignment validation checks based on RAI_PROFILE and your requirements) ...
    return errors




# --- Test Case Scenario ---

def suggest_alternatives(passenger, original_flight):
    """Suggests alternative flights with the same airports and closest departure time."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()

    # Query OAG flights with the same origin and destination:
    cursor.execute("""
        SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_type, capacity
        FROM oag_flights
        WHERE origin = ? AND destination = ? AND status = 'Scheduled'
        ORDER BY ABS(strftime('%s', departure_time) - strftime('%s', ?))  -- Order by time difference
        LIMIT 10  -- Limit to 10 closest flights
    """, (original_flight['origin'], original_flight['destination'], original_flight['departure_time']))  # Pass original departure time

    oag_alternatives = [
        {'flight_number': row[0], 'origin': row[1], 'destination': row[2],
         'departure_time': datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f"),
         'arrival_time': datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f"),
         'aircraft_type': row[5], 'capacity': row[6]}
        for row in cursor.fetchall()
    ]

    # Present the alternatives to the passenger (using print statements for now)
    print("Alternative flights found (OAG):")
    for flight in oag_alternatives:
        print(
            f"  Flight: {flight['flight_number']}, Departure: {flight['departure_time'].strftime('%Y-%m-%d %H:%M')}, Arrival: {flight['arrival_time'].strftime('%Y-%m-%d %H:%M')}"
        )

    conn.close()

def select_flight_and_passenger():
    """Selects a flight to cancel, passenger, and suggests alternatives if needed."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()

    try:
         # 1. Select a flight to cancel (e.g., flight with highest cost):
        cursor.execute("""
            SELECT flight_number, origin, destination, departure_time, arrival_time
            FROM flights
            ORDER BY cost DESC
            LIMIT 1
        """)  # Select by cost and retrieve origin/destination

        flight_data = cursor.fetchone()



        if flight_data:  # Check if flight_data is not None
              cancelled_flight_number, origin, destination, departure_time, arrival_time = flight_data

              original_flight = {
                'flight_number': cancelled_flight_number,
                'origin': origin,
                'destination': destination,
                'departure_time': departure_time,  # Store departure_time as datetime objec
                'arrival_time': datetime.strptime(arrival_time, '%Y-%m-%d %H:%M:%S.%f') # Assuming arrival_time is a string

              }

              # Sanitize values:
              origin = sanitize_input(original_flight['origin'])
              destination = sanitize_input(original_flight['destination'])
              # No need to create departure_time_str
              arrival_time_str = sanitize_input(str(original_flight['arrival_time'] + RAI_PROFILE['connection_times']['domestic']))
              end_time_str = sanitize_input(str(original_flight['arrival_time'] + timedelta(hours=24)))


              # Print flight details:
              print(f"Cancelling flight: {cancelled_flight_number}")
              print(f"Departure Airport: {origin}, Departure Time: {departure_time}")
              print(f"Arrival Airport: {destination}, Arrival Time: {arrival_time}")
              update_flight_status(cancelled_flight_number, "Canceled")
              print('\n')




        else:  # Handle the case where no flight was found
              print("No flights found to cancel.")
              return None, None



        # 2. Select a random passenger from the cancelled flight:
        cursor.execute("SELECT pnr FROM passengers WHERE flight_number = ? ORDER BY RANDOM() LIMIT 1", (cancelled_flight_number,))
        passenger_pnr_result = cursor.fetchone()

        if passenger_pnr_result:
            passenger_pnr = passenger_pnr_result[0]
            print(f"Selected passenger for test case: (PNR: {passenger_pnr}, Flight: {cancelled_flight_number})")
            #return passenger_pnr, cancelled_flight_number  # Return the PNR and flight number

            # Return all required values:
            return (passenger_pnr, original_flight, origin, destination,
                        original_flight['departure_time'], arrival_time_str, end_time_str)  # Return departure_time directly



        else:
            # 3. If no passenger found, suggest alternative flights:
            print(f"No passengers found on the cancelled flight {cancelled_flight_number}. Suggesting alternative flights...")

            # Retrieve original flight data (including origin and departure_time):
            cursor.execute("SELECT origin, destination, departure_time, arrival_time FROM flights WHERE flight_number = ?", (cancelled_flight_number,))
            flight_data = cursor.fetchone()

            if flight_data:
                origin, destination, departure_time_str, arrival_time_str = flight_data
                departure_time = datetime.strptime(departure_time_str, "%Y-%m-%d %H:%M:%S.%f")
                arrival_time = datetime.strptime(arrival_time_str, "%Y-%m-%d %H:%M:%S.%f")

                original_flight = {
                    "flight_number": cancelled_flight_number,
                    "origin": origin,
                    "destination": destination,
                    "departure_time": departure_time,  # Include departure time
                    "arrival_time": arrival_time
                }

                # Suggest alternative flights using the updated suggest_alternatives function:
                suggest_alternatives(None, original_flight)  # Passenger is None as no passenger found
            else:
                print("Original flight data not found.")
                print(f"RecoverAI Agent: Original flight data not found for flight {cancelled_flight_number}. Cannot proceed with rebooking.")

            return None, None

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: An error occurred: {e}")
        return None, None  # Return None in case of an error
    finally:
        conn.close()


def run_test_case(passenger_pnr, original_flight):
    """Runs a test case for the RecoverAI agent."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()

    try:
        # Get passenger details using PNR
        cursor.execute("SELECT * FROM passengers WHERE pnr = ?", (passenger_pnr,))
        passenger = cursor.fetchone()

        if passenger:
            print(f"Passenger details: {passenger}")

            # Display original flight details (using original_flight)
            print(f"Original Flight: {original_flight['flight_number']} from {original_flight['origin']} to {original_flight['destination']}")

            # Call find_alternative_flight using passenger and original_flight
            alternative_flight = find_alternative_flight(passenger, original_flight)

            if alternative_flight:
                update_passenger_booking(passenger, alternative_flight)
                generate_notification(passenger, alternative_flight)
            else:
                print("RecoverAI Agent: No suitable alternatives found for passenger.")

        else:
            print(f"RecoverAI Agent: Passenger with PNR '{passenger_pnr}' not found.")

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: Error during test case: {e}")

    finally:
        conn.close()  # Always close the connection

def run_test_case_old(passenger_pnr, original_flight):
#def run_test_case(passenger_pnr, cancelled_flight_number)
    """Executes the RecoverAI flight cancellation and rebooking test case."""
    try:
        conn = sqlite3.connect('recoverai.db')
        cursor = conn.cursor()



        # RecoverAI Actions (Rebooking)
        # Retrieve the passenger's data using the provided passenger_pnr and cancelled_flight_number:
        cursor.execute("SELECT * FROM passengers WHERE pnr = ? AND flight_number = ?", (passenger_pnr, cancelled_flight_number))
        passenger = cursor.fetchone()

        if passenger:
           # Get the original flight's destination and arrival_time:
            cursor.execute("SELECT destination, arrival_time FROM flights WHERE flight_number = ?", (cancelled_flight_number,))
            flight_data = cursor.fetchone()

            if flight_data:
                destination, arrival_time_str = flight_data
                arrival_time = datetime.strptime(arrival_time_str, "%Y-%m-%d %H:%M:%S.%f")  # Convert arrival_time to datetime object


                original_flight = {
                    "flight_number": cancelled_flight_number,
                    "destination": destination,
                    "arrival_time": arrival_time
                }

            # Find alternative flight
            alternative_flight = find_alternative_flight(passenger, original_flight)

            if alternative_flight:
                # Update passenger booking
                update_passenger_booking(passenger, alternative_flight)
            else:
                print(f"RecoverAI Agent: No suitable alternative found for {passenger[1]}.")  # Access passenger name from the tuple
        else:
            print(f"Passenger with PNR {passenger_pnr} not found on the cancelled flight {cancelled_flight_number}.")

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: An error occurred: {e}")
    finally:
        if conn:
            conn.close()


    conn.close()


# --- Main Execution ---
if __name__ == "__main__":

    print('\n')
    print("RecoverAI Agent: Creating tables...")
    create_tables()

    print('\n')
    print("RecoverAI Agent: Generating data...")
    generate_data()

    print('\n')
    print("RecoverAI Agent: Running test case scenario...")
    print('\n')

    (passenger_pnr, original_flight, origin, destination, departure_time,
    arrival_time_str, end_time_str) = select_flight_and_passenger()

    if passenger_pnr and original_flight:  # Use original_flight, not cancelled_flight_number
        print(f"Testing recovery for passenger with PNR: {passenger_pnr}")
        # ... (Your recovery logic using passenger_pnr, original_flight, and other returned values) ...
        run_test_case(passenger_pnr, original_flight)  # Assuming this function now needs original_flight info
    else:
        print("Could not select a flight and passenger for the test case.")


RecoverAI Agent: Existing database file 'recoverai.db' deleted.


RecoverAI Agent: Creating tables...


RecoverAI Agent: Generating data...


NUMBER OF GLOBAL RECORDS

In [3]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('recoverai.db')
cursor = conn.cursor()

# Get the number of records in each table
tables = ['passengers', 'flights', 'seat_assignments', 'crew', 'aircraft', 'oag_flights']

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"Number of records in {table}: {count}")

# Close the connection
conn.close()

Number of records in passengers: 500
Number of records in flights: 100
Number of records in seat_assignments: 500
Number of records in crew: 200
Number of records in aircraft: 50
Number of records in oag_flights: 200


In [4]:
import sqlite3

def find_missing_passenger():
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()

    cursor.execute("""
        SELECT p.passenger_id, p.name, p.flight_number
        FROM passengers p
        LEFT JOIN seat_assignments sa ON p.passenger_id = sa.passenger_id
        WHERE sa.passenger_id IS NULL;
    """)

    missing_passenger = cursor.fetchone()
    conn.close()

    if missing_passenger:
        print(f"Missing passenger: {missing_passenger}")
        passenger_id, name, flight_number = missing_passenger
        investigate_missing_assignment(passenger_id, name, flight_number)
    else:
        print("All passengers have seat assignments.")

def investigate_missing_assignment(passenger_id, name, flight_number):
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()

    # 1. Check aircraft capacity
    cursor.execute("""
        SELECT T2.type, T2.capacity
        FROM flights AS T1
        INNER JOIN aircraft AS T2 ON T1.aircraft_id = T2.aircraft_id
        WHERE T1.flight_number = ?
    """, (flight_number,))
    aircraft_data = cursor.fetchone()

    if aircraft_data:
        aircraft_type, capacity = aircraft_data
        print(f"Passenger {name} is on flight {flight_number} with aircraft type {aircraft_type} (capacity: {capacity}).")

        # Check available seats in the passenger's class of service (you'll need to add logic for this)
        # ...

    else:
        print(f"Flight {flight_number} does not have a matching aircraft type.")

    # 2. Check flight data (ensure valid aircraft_id)
    # ... (Add logic to check flight data)

    # 3. Check error logs (if you have any)
    # ... (Add logic to check error logs)

    conn.close()

if __name__ == "__main__":
    find_missing_passenger()

All passengers have seat assignments.


10 FIRST RECORDS PER TABLES

In [5]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('recoverai.db')
cursor = conn.cursor()

# Get the first 3 records in each table
tables = ['passengers', 'flights', 'oag_flights', 'seat_assignments', 'crew', 'aircraft']

for table in tables:
    cursor.execute(f"SELECT * FROM {table} LIMIT 10")
    records = cursor.fetchall()  # Fetch all 3 records

    print(f"\nFirst 10 records in {table}:")
    for record in records:
        print(record)

# Close the connection
conn.close()


First 10 records in passengers:
(1, 'Ronnie Scott', 'SA002', None, 'gODMG-07155', 'EC', 'R')
(2, 'Emily Garcia', 'SA081', None, 'tBhUL-24251', 'BC', 'R')
(3, 'Jessica Garza', 'SA068', None, 'VzraC-41382', 'EC', 'R')
(4, 'Joshua Wagner', 'SA001', None, 'oAIhB-32390', 'EC', 'R')
(5, 'Madison Smith', 'SA037', None, 'dhcsF-29569', 'BC', 'R')
(6, 'Melissa Patel', 'SA052', None, 'SbOUs-22381', 'EC', 'R')
(7, 'Henry Gonzales', 'SA024', None, 'KwydU-56003', 'EC', 'R')
(8, 'Justin Thompson', 'SA051', None, 'JPqKW-24405', 'EC', 'R')
(9, 'Robert Scott', 'SA073', None, 'PuaOu-26213', 'BC', 'R')
(10, 'Kristine James', 'SA042', None, 'eAkYX-62845', 'EC', 'R')

First 10 records in flights:
('SA001', 'CUN', 'CDG', '2025-02-01 17:15:41.499026', '2025-02-02 05:15:41.499026', 'B787-035', 'Scheduled', 9948.15, 8610.00529255663)
('SA002', 'AMS', 'DFW', '2025-01-30 22:37:48.435867', '2025-01-31 03:37:48.435867', 'A320-044', 'Scheduled', 2809.6, 13396.286716007939)
('SA003', 'PVG', 'CDG', '2025-01-30 23:22: